<h2 style="color: red">UNTUK MELIHAT HASIL (SISTEM "SAMPAH DI BANDUNG") ADA DI CELL PALING BAWAH</h2>


In [37]:
# INSTALL LIBRARY # LIBRARY YANG KAMI PAKAI

# pip install folium numpy pandas geopandas matplotlib ipywidgets

import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import *

%store -r data_sampah
%store -r total_sampah_per_tahun
%store -r top_5_data_banyak
%store -r top_5_data_sedikit
%store -r min_penigkatan_sorted
%store -r top_5_kecamatan_signifikan
%store -r koordinat_BDG

In [38]:
#Scrapping# DATA KECAMATAN
data_kecamatan = pd.read_html('https://id.wikipedia.org/wiki/Daftar_kecamatan_dan_kelurahan_di_Kota_Bandung')
data_kecamatan[0]

kecamatannya = data_kecamatan[0]
ndf = kecamatannya['Kecamatan']
kecamatan_fix = ndf.iloc[:-1]

sampah_sampah = [1,2] #tingkatan dari jenis sampah


data = {
    'id': pd.Series([i for i in range(0,2891)]),
    'tanggal' : pd.date_range('2016-01-01', '2023-11-30'),
    'kecamatan' : pd.Series(),
    'jenis_sampah' : pd.Series(),
    'jumlah_sampah' : pd.Series(),
    'satuan' : pd.Series()
}

df = pd.DataFrame(data)

df['kecamatan'] = df['kecamatan'].apply(lambda x: kecamatan_fix[np.random.choice(len(kecamatan_fix))])

df['jenis_sampah'] = df['jenis_sampah'].apply(lambda x : sampah_sampah[np.random.choice(len(sampah_sampah))])

tonase = [np.random.randint(1, 10) for i in range(len(df))]

df['jumlah_sampah'] = tonase

df['satuan'] = df['satuan'].fillna('TON')

# df
#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [39]:
#Scrapping# JENIS SAMPAH

pd.set_option('display.max_colwidth', None)

# scrapping
url = 'https://bandungkota.bps.go.id/statictable/2021/03/23/1457/produksi-sampah-menurut-jenisnya-di-kota-bandung-2020.html'
jenis_sampah = pd.read_html(url)

# Mengambil data dari Tabel 6 yang ada dilink diatas
data_produksi_sampah = jenis_sampah[5]

# Mengambil data
jenis_sampah = data_produksi_sampah.iloc[4:14, 0]
produksi_sampah = data_produksi_sampah.iloc[6:15, 1]
persentase = data_produksi_sampah.iloc[6:15, 2]

# nyatuin kolom-kolom jadi satu DataFrame
df = pd.concat([jenis_sampah, produksi_sampah, persentase], axis=1)
df.columns = ['Jenis Sampah', 'Produksi Sampah', 'Persentase']

# Menghilangkan baris yang ada nilai NaN
data_jenis_sampah = df.dropna()

# Menghilangkan Kata setelah '/'
data_jenis_sampah.loc[:, 'Jenis Sampah'] = data_jenis_sampah['Jenis Sampah'].str.split('/').str[0].str.strip()

# display(data_jenis_sampah)
#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [40]:
#DataFrame# DATA ASLI

data_sampah = pd.read_csv('../datasets/Dataset_dummy_v2.csv')
data_sampah['tanggal'] = pd.to_datetime(data_sampah['tanggal'])

# display(data_sampah)
#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [41]:
#DataFrame# JUMLAH SAMPAH PER TAHUN 

data_sampah['tanggal'] = pd.to_datetime(data_sampah['tanggal'])

# Tambahkan kolom 'tahun' dengan menggunakan .dt.year
data_sampah['tahun'] = data_sampah['tanggal'].dt.year

# Konversi jumlah_sampah ke ton (asumsi awal dalam kilogram)
data_sampah['jumlah_sampah_ton'] = data_sampah['jumlah_sampah'] / 1000

# Agregasi jumlah sampah per tahun
total_sampah_per_tahun = data_sampah.groupby('tahun')['jumlah_sampah_ton'].sum().reset_index()


# display(total_sampah_per_tahun)
#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [42]:
#DataFrame# 5 KECAMATAN DENGAN JUMLAH SAMPAH TERBANYAK

data_sampah_grouped = data_sampah.groupby('kecamatan')['jumlah_sampah'].sum()

top_5_data_banyak = pd.DataFrame(data_sampah_grouped).sort_values(by='jumlah_sampah', ascending=False).head(5)

# display(top_5_data_banyak)
#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [43]:
#DataFrame# 5 KECAMATAN DENGAN JUMLAH SAMPAH TERSEDIKIT

# Mengelompokkan dan menghitung jumlah sampah per kecamatan
data_sampah_grouped = data_sampah.groupby('kecamatan')['jumlah_sampah'].sum()

# Membuat DataFrame baru dengan kolom 'kecamatan' dan 'jumlah_sampah'
top_5_data_sedikit = pd.DataFrame(data_sampah_grouped).sort_values(by='jumlah_sampah', ascending=True).head(5)

# display(top_5_data_sedikit)
#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [44]:
#DataFrame# 5 KECAMATAN DENGAN PENINGKATAN SAMPAH MINIM

data_sampah['tanggal'] = pd.to_datetime(data_sampah['tanggal'])

data_sampah['tahun'] = data_sampah['tanggal'].dt.year

grouping = data_sampah.groupby(['kecamatan', 'tahun'])['jumlah_sampah'].sum().reset_index()

grouping['peningkatan_sampah'] = grouping.groupby('kecamatan')['jumlah_sampah'].diff()

min_peningkatan = grouping.loc[grouping.groupby('kecamatan')['peningkatan_sampah'].idxmin()]

min_penigkatan_sorted = min_peningkatan.sort_values(by='peningkatan_sampah').head(5)

# display(min_penigkatan_sorted)
#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [45]:
#DataFrame# 5 KECAMATAN DENGAN KENAIKAN PRODUKSI SAMPAH YANG SIGNIFIKAN

data_sampah['tanggal'] = pd.to_datetime(data_sampah['tanggal'])
data_sampah['tahun'] = data_sampah['tanggal'].dt.year

# Mengelompokkan data berdasarkan kecamatan dan tahun
grouping = data_sampah.groupby(['kecamatan', 'tahun'])['jumlah_sampah'].sum().reset_index()

# Menghitung peningkatan sampah per tahun
grouping['peningkatan_sampah'] = grouping.groupby('kecamatan')['jumlah_sampah'].diff()

# Menentukan 5 kecamatan dengan peningkatan paling signifikan pertahunnya
kecamatan_tertinggi = grouping.loc[grouping.groupby('kecamatan')['peningkatan_sampah'].idxmax()]

# Memilih hanya 5 kecamatan tertinggi
top_5_kecamatan_signifikan = kecamatan_tertinggi.nlargest(5, 'peningkatan_sampah')

# display(top_5_kecamatan_signifikan)
#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [46]:
#VISUALISASI# JUMLAH SAMPAH PER TAHUN

# fig, ax = plt.subplots(figsize=(18, 10))
# ax.plot(total_sampah_per_tahun['tahun'], total_sampah_per_tahun['jumlah_sampah_ton'], marker='o')

# plt.style.use('_mpl-gallery') 
# plt.title('Grafik Total Jumlah Sampah per Tahun')
# plt.xlabel('Tahun')
# plt.ylabel('Jumlah Sampah (TON)')
# plt.grid(True)

# plt.show()


#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [47]:
#VISUALISASI# 5 KECAMATAN DENGAN KENAIKAN PRODUKSI SAMPAH YANG SIGINIFIKAN
# from matplotlib.patches import Patch

# fig, ax = plt.subplots(figsize=(12, 6))

# bar_colors = []
# legend_colors = ['red', 'orange', 'green']
# legend_labels = ['Meledak', 'Membludak', 'Menumpuk']

# for idx, row in top_5_kecamatan_signifikan.iterrows():
#     if row['peningkatan_sampah'] > 500:
#         bar_colors.append('red')

#     elif row['peningkatan_sampah'] > 200:
#         bar_colors.append('orange')

#     else:
#         bar_colors.append('green')

# info = [Patch(color=color, label=label) for color, label in zip(legend_colors, legend_labels)]

# ax.bar(top_5_kecamatan_signifikan['kecamatan'], top_5_kecamatan_signifikan['peningkatan_sampah'], color=bar_colors)

# ax.set_title('5 Kecamatan dengan Peningkatan Sampah Paling Signifikan pertahunnya')
# ax.set_xlabel('Kecamatan')
# ax.set_ylabel('Peningkatan Sampah (TON)')
# plt.legend(handles=info)

# plt.show()

#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [48]:
#VISUALISASI# JENIS SAMPAH

# data_jenis_sampah['Persentase'] = data_jenis_sampah['Persentase'].astype(str).str.replace(',', '.').str.rstrip('%')
# data_jenis_sampah.loc[:, 'Persentase'] = data_jenis_sampah['Persentase'].astype('float')

# jenis_sampah = data_jenis_sampah.copy()

# labels = jenis_sampah['Jenis Sampah']
# sizes = jenis_sampah['Persentase']

# # Mengatur ukuran figure
# plt.figure(figsize=(10, 10))

# # Plotting the pie chart
# plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=45, wedgeprops=dict(edgecolor='w'))

# # Aspect ratio ensures that the pie is drawn as a circle
# plt.axis('equal')

# # Menambahkan judul
# plt.title('Distribusi Jenis Sampah')

# # Menampilkan pie chart
# plt.show()


#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [49]:
#VISUALISASI# KECAMATAN YANG PRODUKSI SAMPAH LEBIH DARI 5000


# filtered_data = data_jenis_sampah[data_jenis_sampah['Produksi Sampah'].astype(float) > 5000]

# # Create a bar chart
# plt.figure(figsize=(12, 6))
# plt.bar(filtered_data['Jenis Sampah'], filtered_data['Produksi Sampah'].astype(float))

# # Add labels and title
# plt.xlabel('Jenis Sampah')
# plt.ylabel('Produksi Sampah (m³)')
# plt.title('Jenis Sampah dengan Produksi Lebih dari 5000 (Tahun 2020)')

# # Rotate x-axis labels for better readability
# plt.xticks(rotation=45, ha='right')

# # Show the plot
# plt.show()


#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [50]:
#VISUALISASI# 5 KECAMATAN DENGAN KENAIKAN SAMPAH TERBANYAK PER TAHUNNYA

# top_5_data_banyak['jumlah_sampah'] = top_5_data_banyak['jumlah_sampah'] / 1000
# fig, ax = plt.subplots(figsize=(10, 6))
# bars = ax.bar(top_5_data_banyak.index, top_5_data_banyak['jumlah_sampah'], color='skyblue')

# # Add labels and title
# ax.set_xlabel('Kecamatan')
# ax.set_ylabel('Jumlah Sampah (Ton)')
# ax.set_title('Top 5 Kecamatan dengan Kenaikan Sampah Terbanyak per Tahunnya')

# # Add values on top of the bars
# for bar in bars:
#     yval = bar.get_height()
#     plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# # Show the plot
# plt.show()

#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [51]:
#VISUALISASI# 5 KECAMATAN DENGAN KENAIKAN SAMPAH TERSEDIKIT PER TAHUNNYA

# top_5_data_sedikit['jumlah_sampah'] = top_5_data_sedikit['jumlah_sampah'] / 1000
# # Create a bar chart
# fig, ax = plt.subplots(figsize=(10, 6))
# bars = ax.bar(top_5_data_sedikit.index, top_5_data_sedikit['jumlah_sampah'], color='skyblue')

# # Add labels and title
# ax.set_xlabel('Kecamatan')
# ax.set_ylabel('Jumlah Sampah (Ton)')
# ax.set_title('Top 5 Kecamatan dengan Kenaikan Sampah Tersedikit per Tahunnya')

# # Add values on top of the bars
# for bar in bars:
#     yval = bar.get_height()
#     plt.text(bar.get_x() + bar.get_width()/2, yval, round(yval, 2), ha='center', va='bottom')

# # Show the plot
# plt.show()

#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [52]:
# GIS # MAP KECAMATAN BANDUNG

# Import data
data_sampah = pd.read_csv('../datasets/Dataset_dummy_v2.csv')
koordinat_BDG = pd.read_csv('../datasets/Koordinat_kecamatan_kota_bandung.csv')
bandung = gpd.read_file('../datasets/kecBandung.json')

# Mengubah kolom tanggal menjadi tipe data datetime
data_sampah['tanggal'] = pd.to_datetime(data_sampah['tanggal'], format='%m/%d/%Y')

# Membuat kolom baru untuk mengelompokkan kolom tanggal berdasarkan tahun
data_sampah['tahun'] = data_sampah['tanggal'].dt.to_period('Y')

# Mengelompokkan berdasarkan tahun dan kecamatan, kemudian menghitung jumlah sampah
grup_tahun_kecamatan = data_sampah.groupby(['tahun', 'kecamatan']).agg({
    'jumlah_sampah': 'sum'
}).reset_index()


data_gabung = pd.merge(grup_tahun_kecamatan, koordinat_BDG, on='kecamatan')

# Memasukan longitude dan latitud (rata-rata supaya berada di tengah)
bandung_map = folium.Map(location=[koordinat_BDG['Lintang Selatan'].mean(), koordinat_BDG['Bujur Timur'].mean()], zoom_start=13)


for index, row in data_gabung.iterrows():
    # Menentukan warna marker berdasarkan jumlah sampah
    if row['jumlah_sampah'] > 1500:
        color = 'red'
    elif row['jumlah_sampah'] > 1000:
        color = 'orange'
    else:
        color = 'green'

    # Membuat HTML untuk Tooltip
    tooltip = f"""
    <h4><strong>Kec.{row['kecamatan']}</strong></h4>
    <h5><strong>Tahun:</strong> 2016 - 2023</h5>
    <h5><strong>Jumlah Sampah:</strong> {row['jumlah_sampah']} Ton</h5>
    """

    # Menambahkan marker untuk setiap baris dengan warna berdasarkan jumlah sampah
    marker = folium.Marker(
        location=[row['Lintang Selatan'], row['Bujur Timur']],
        tooltip=tooltip,
        icon=folium.Icon(color=color),
    )
    marker.add_to(bandung_map)


bandung_geojson = bandung.to_json()
folium.GeoJson(bandung_geojson).add_to(bandung_map)
# Menampilkan peta di notebook
# display(bandung_map)

#DICOMMENT KARENA DI TAMPILKAN DI MAIN LAYOUT

In [53]:
# MAIN LAYOUT # SISTEM SAMPAH DI BANDUNG (ISI TUGAS BESAR)


title = widgets.HTML('<h1 style="text-align: center;">PENDATAAN SAMPAH KECAMATAN DI BANDUNG</h1>')

# ---------------------------------------------------------------------------------------------------------------------------------------------------- #

#Canvas untuk tampilan content
output = widgets.Output(layout={'width': '900px','height':'800px'})


with output :
        halo = widgets.HTML('<div style="margin-left:130px; font-size: 18px; color: cyan;"><b>Selamat Datang, Silahkan Klik Salah Satu Button dikiri ^.^</b></div>')
        display(halo)

# ---------------------------------------------------------------------------------------------------------------------------------------------------- #

#Bikin buton buat navigasi ke beberapa data

button_sampah = widgets.Button(
    description='Data Sampah',
    disabled=False,
    button_style='info',
    tooltip='Open the Datasets',
    icon='database'
)

button_bar = widgets.Button(
    description='Diagram Sampah',
    button_style='warning',
    tooltip='Open the Bar',
    icon='bar-chart'
)

button_map = widgets.Button(
    description='Lihat Map',
    button_style='success',
    tooltip='Open the Map',
    icon='map'
)

button_close = widgets.Button(
    description='Close',
    button_style='danger',
    tooltip='Close',
    icon='times'
)


# ---------------------------------------------------------------------------------------------------------------------------------------------------- #


def open_sampah(button_sampah):
    with output:
        clear_output()
        tabel_aja = widgets.Output(layout={'width': '100%'})

        options = ['Data Sampah di tiap Kecamatan','Jenis Sampah','Total Sampah Tahun 2016 - 2023','5 Kecamatan dengan produksi sampah yang Signifikan','5 Kecamatan dengan peningkatan Sampah yang Minim', '5 Kecamatan dengan Sampah ter Sedikit', '5 Kecamatan dengan Sampah ter Banyak']
        dropdown = widgets.Dropdown(options=options, description='Pilih Dataset :', value=options[0])
        
        event_atas = widgets.VBox([dropdown, button_close], layout=widgets.Layout(display='flex', flex_flow='row', align_items='baseline', justify_content='space-between'))
        konten_tabel = widgets.HBox([tabel_aja], layout=widgets.Layout(padding='10px 20px 20px 90px')) 

        tampil_dataset  = widgets.VBox([event_atas, widgets.HBox([konten_tabel])])

        display(tampil_dataset)

        
        with tabel_aja :
            judul = widgets.HTML('<h2>Data Sampah di tiap Kecamatan</h2>')
            display(judul)
            display(data_sampah)
            
        def display_table(change):
            with tabel_aja:
                clear_output()
                selected_option = change['new']
                if selected_option == 'Data Sampah di tiap Kecamatan' :
                    judul = widgets.HTML('<h2>Data Sampah di tiap Kecamatan</h2>')
                    display(judul)
                    display(data_sampah)
                    
                elif selected_option == 'Total Sampah Tahun 2016 - 2023':
                    judul = widgets.HTML('<h2>Total Sampah Tahun 2016 - 2023</h2>')
                    display(judul)
                    display(total_sampah_per_tahun)
                    
                elif selected_option == '5 Kecamatan dengan Sampah ter Banyak':
                    judul = widgets.HTML('<h2>5 Kecamatan dengan Sampah ter Banyak</h2>')
                    display(judul)
                    display(top_5_data_banyak)
                    
                elif selected_option == '5 Kecamatan dengan Sampah ter Sedikit':
                    judul = widgets.HTML('<h2>5 Kecamatan dengan Sampah ter Sedikit</h2>')
                    display(judul)
                    display(top_5_data_sedikit)
                    
                elif selected_option == '5 Kecamatan dengan peningkatan Sampah yang Minim':
                    judul = widgets.HTML('<h2>5 Kecamatan dengan peningkatan Sampah yang Minim</h2>')
                    display(judul)
                    display(min_penigkatan_sorted)
                    
                elif selected_option == '5 Kecamatan dengan produksi sampah yang Signifikan':
                    judul = widgets.HTML('<h2>5 Kecamatan dengan produksi sampah yang Signifikan</h2>')
                    display(judul)
                    display(top_5_kecamatan_signifikan)
                    
                elif selected_option == 'Jenis Sampah':
                    judul = widgets.HTML('<h2>Jenis Sampah yang ada di Kota Bandung</h2>')
                    display(judul)
                    display(data_jenis_sampah)
        
        dropdown.observe(display_table, names='value')


# ---------------------------------------------------------------------------------------------------------------------------------------------------- #


def open_bar(button_bar):
    with output:
        clear_output()
        diagram_aja = widgets.Output(layout={'width': '800px'})
   

        options = ['Pilih Diagram','Jenis Sampah','Jenis Sampah dengan produksi lebih dari 5000','Total Sampah Tahun 2016 - 2023','5 Kecamatan dengan produksi sampah yang Signifikan', '5 Kecamatan dengan Sampah ter Sedikit', '5 Kecamatan dengan Sampah ter Banyak']
        dropdown = widgets.Dropdown(options=options, description='Pilih Diagram :', value=options[0])

        event_atas = widgets.VBox([dropdown, button_close], layout=widgets.Layout(display='flex', flex_flow='row', align_items='baseline', justify_content='space-between'))
        konten_diagram = widgets.HBox([diagram_aja], layout=widgets.Layout(padding='10px 20px 20px 0'))

        tampil_dataset  = widgets.VBox([event_atas, widgets.HBox([konten_diagram])])
        display(tampil_dataset)
        with diagram_aja:
            display(Image(filename='../images/chart.png'))

        def display_table(change):
            with diagram_aja:
                clear_output(wait=True)  # Clear the output and wait for new content
                selected_option = change['new']
                if selected_option == 'Total Sampah Tahun 2016 - 2023':
                    display(Image(filename='../images/data_sampah_pertahun.png', width=912, height=530))
                elif selected_option == '5 Kecamatan dengan Sampah ter Banyak':
                    display(Image(filename='../images/top-5-banyak.png',width=646, height=420))
                elif selected_option == '5 Kecamatan dengan Sampah ter Sedikit':
                    display(Image(filename='../images/top-5-sedikit.png',width=646, height=420))
                elif selected_option == '5 Kecamatan dengan produksi sampah yang Signifikan':
                    display(Image(filename='../images/top-5-signifikan.png',width=773, height=420))
                elif selected_option == 'Jenis Sampah':
                    display(Image(filename='../images/jenis_sampah_chart.png',width=618, height=541))
                elif selected_option == 'Jenis Sampah dengan produksi lebih dari 5000':
                    display(Image(filename='../images/jenis_sampah_5000.png',width=682, height=443))

        dropdown.observe(display_table, names='value')

# ---------------------------------------------------------------------------------------------------------------------------------------------------- #

def open_map(button_map):
    with output:
        clear_output()
        display(button_close, bandung_map)

# ---------------------------------------------------------------------------------------------------------------------------------------------------- #

def close(button_close):
    with output:
        clear_output()
        halo = widgets.HTML('<div style="margin-left:130px; font-size: 18px; color: cyan;"><b>Selamat Datang, Silahkan Klik Salah Satu Button dikiri ^.^</b></div>')
        display(halo)
# ---------------------------------------------------------------------------------------------------------------------------------------------------- #

#Menjalankan Function 
button_map.on_click(open_map)
button_bar.on_click(open_bar)
button_sampah.on_click(open_sampah)
button_close.on_click(close)

# Judul atas
title_box = widgets.VBox([title],layout=widgets.Layout(margin='20px 0 40px 0'))

# Sidebar untuk butong kiri 
sidebar = widgets.VBox([button_sampah, button_bar,button_map])

# Konten untuk nampilin tampilan
content = widgets.HBox([output], layout=widgets.Layout(margin='0 0 0 20px'))  # Adjust the margin values as needed

# Tampilan Utama (Seluruh)
main_layout = widgets.VBox([title_box, widgets.HBox([sidebar, content])])


display(main_layout)
